In [7]:
import numpy as np
from netCDF4 import Dataset
from datetime import datetime, timedelta
import os
import glob


In [3]:
# 設置文件夾保存圖像
output_folder = "ASCII_out"
os.makedirs(output_folder, exist_ok=True)

In [14]:
# 設定範圍和條件
# 將日期定義為變量
date = "2020-02-05"
dtype = "test_data"

# 定义经纬度范围
lon1, lon2 = 120, 130
lat1, lat2 = 10, 20

In [16]:
# 打开 NetCDF 文件
# 使用 glob 查找對應檔案
file = f"{dtype}/RSS_AMSR2_ocean_L3_daily_{date}_v08.2.nc"
nc = Dataset(file, mode='r')

# 讀取變量
time = nc.variables['time'][:]
sst = nc.variables['SST'][:]
wspdLF = nc.variables['wind_speed_LF'][:]
wspdMF = nc.variables['wind_speed_MF'][:]
vapor = nc.variables['water_vapor'][:]
cloud = nc.variables['cloud_liquid_water'][:]
rain = nc.variables['rain_rate'][:]
land_mask = nc.variables['land_mask'][:]
ice_mask = nc.variables['sea_ice_mask'][:]
coast_mask = nc.variables['coast_mask'][:]
noobs_mask = nc.variables['noobs_mask'][:]

# 提取日期部分
parts = file.split('_')
for part in parts:
    if len(part) == 10:
        try:
            base_date = datetime.strptime(part, '%Y-%m-%d')
            print(f"Extracted date: {base_date}")
            break
        except ValueError:
            continue
else:
    raise ValueError("No valid date found in the file name.")

# 定義經緯度範圍
lon1, lon2 = 120, 130
lat1, lat2 = 10, 20

# 經緯度網格
latx = 0.25 * (np.arange(1, 721)) - 90.125
lonx = 0.25 * (np.arange(1, 1441)) - 0.125
lon, lat = np.meshgrid(lonx, latx)

# 查找感興趣區域的索引
ix = np.where((lonx >= lon1) & (lonx <= lon2))[0]
iy = np.where((latx >= lat1) & (latx <= lat2))[0]

# 輸出文件名包含日期
output_file_ascending = os.path.join(output_folder, f"ascending_data_{base_date.strftime('%Y%m%d')}.dat")
output_file_descending = os.path.join(output_folder, f"descending_data_{base_date.strftime('%Y%m%d')}.dat")

# 輸出到文件
with open(output_file_ascending, 'w') as fid1, open(output_file_descending, 'w') as fid2:
    # 寫入標題
    title = "Time, Longitude (E), Latitude (N), SST, WSPD_LF, WSPD_MF, Vapor, Cloud_Liquid_Water, Rain_Rate\n"
    fid1.write(title)
    fid2.write(title)
    print("File headers written.")

    # 確保 lon[y] 和 lat[x] 是標量值
    for x in ix:
        for y in iy:
            longitude = lon[y, x]
            latitude = lat[y, x]

            # 上行數據（ascending）
            if time[0, y, x] <= 250:  # 檢查條件
                jdaygmt = base_date + timedelta(hours=float(time[0, y, x]))
                fid1.write(
                    f"{jdaygmt:%Y-%m-%d %H:%M:%S}, {longitude:.3f}, {latitude:.3f}, "
                    f"{sst[0, y, x]:.3f}, {wspdLF[0, y, x]:.3f}, {wspdMF[0, y, x]:.3f}, "
                    f"{vapor[0, y, x]:.3f}, {cloud[0, y, x]:.3f}, {rain[0, y, x]:.3f}\n"
                )

            # 下行數據（descending）
            if time[1, y, x] <= 250:  # 檢查條件
                jdaygmt = base_date + timedelta(hours=float(time[1, y, x]))
                fid2.write(
                    f"{jdaygmt:%Y-%m-%d %H:%M:%S}, {longitude:.3f}, {latitude:.3f}, "
                    f"{sst[1, y, x]:.3f}, {wspdLF[1, y, x]:.3f}, {wspdMF[1, y, x]:.3f}, "
                    f"{vapor[1, y, x]:.3f}, {cloud[1, y, x]:.3f}, {rain[1, y, x]:.3f}\n"
                )
    print("Data written to files successfully.")

# 檢查輸出文件
print(f"Ascending data file: {output_file_ascending}")
print(f"Descending data file: {output_file_descending}")

Extracted date: 2020-02-05 00:00:00
File headers written.
Data written to files successfully.
Ascending data file: ASCII_out/ascending_data_20200205.dat
Descending data file: ASCII_out/descending_data_20200205.dat


/var/folders/lx/4bn5cmyn63b1p7gtb6m71jch0000gn/T/ipykernel_97165/2830380820.py:67: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  f"{jdaygmt:%Y-%m-%d %H:%M:%S}, {longitude:.3f}, {latitude:.3f}, "
/var/folders/lx/4bn5cmyn63b1p7gtb6m71jch0000gn/T/ipykernel_97165/2830380820.py:76: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  f"{jdaygmt:%Y-%m-%d %H:%M:%S}, {longitude:.3f}, {latitude:.3f}, "
